Importações

In [43]:
import pandas as pd
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Carregando os dados

In [44]:

caminho = 'excel/viajou-tratado copy.xlsx'
df = pd.read_excel(caminho)
df

,idade,genero,municipio_residencia,visita_pontos_turisticos,participa_eventos,frequencia_eventos,tipos_eventos_preferidos,participa_excursões,frequencia_tours_virtuais,frequencia_apps_turismo,usou_apps_turismo,confianca_avaliacoes,interesse_acessibilidade,pagaria_por_tour_virtual,descoberta_eventos_atracoes,motivacao_uso_app_turismo,probabilidade_recomendar
0,Menor de 18 anos,Feminino,São Paulo,Sim,Sim,Ocasionalmente,['Teatrais'],Não,Nunca,Ocasionalmente,Sim,3,"Sim, mas não é essencial","Não, prefiro gastar em outra coisa",['Redes Sociais'],Encontrar Novas Experiências,3
1,Menor de 18 anos,Masculino,São Paulo,Não,Sim,Ocasionalmente,"['Eletrônicos', 'Gastronômicos', 'Musicais']",Não,Nunca,Raramente,Sim,5,"Sim, mas não é essencial","Sim, se o preço for acessível","['Recomendações de Amigos e Família', 'Redes S...",Obter Informações Detalhadas,4
2,Menor de 18 anos,Feminino,São Paulo,Não,Não,Raramente,['Gastronômicos'],Não,Nunca,Raramente,Sim,4,"Sim, mas não é essencial","Sim, se o preço for acessível","['Recomendações de Amigos e Família', 'Redes S...",Encontrar Novas Experiências,4
3,Mais de 55 anos,Feminino,São Paulo,Sim,Não,Ocasionalmente,"['Culturais', 'Musicais']",Não,Raramente,Ocasionalmente,Sim,3,"Sim, é muito importante pra mim.","Sim, se o preço for acessível",['Agências de Viagem e Operadoras de Turismo'],Obter Informações Detalhadas,3
4,Menor de 18 anos,Outro,Osasco,Sim,Sim,Ocasionalmente,"['Culturais', 'Musicais', 'Teatrais']",Sim,Raramente,Raramente,Não,4,"Sim, é muito importante pra mim.","Talvez, depende da situação","['Recomendações de Amigos e Família', 'Redes S...",Encontrar Novas Experiências,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Mais de 55 anos,Masculino,São Paulo,Não,Não,Raramente,"['Esportivos', 'Gastronômicos']",Não,Nunca,Nunca,Não,4,"Sim, mas não é essencial","Não, prefiro gastar em outra coisa",['Recomendações de Amigos e Família'],Receber Recomendações Personalizadas,3
109,35-44 anos,Masculino,Osasco,Não,Sim,Mensalmente,"['Esportivos', 'Gastronômicos']",Sim,Raramente,Sempre que saio.,Sim,4,"Sim, é muito importante pra mim.","Sim, se o preço for acessível",['Aplicativos de Turismo'],Reservar Ingressos e Excursões,1
110,25-34 anos,Feminino,São Paulo,Sim,Sim,Ocasionalmente,"['Eletrônicos', 'Esportivos', 'Gastronômicos',...",Não,Ocasionalmente,Ocasionalmente,Sim,4,"Sim, mas não é essencial","Não, prefiro gastar em outra coisa","['Recomendações de Amigos e Família', 'Redes S...",Reservar Ingressos e Excursões,2
111,Mais de 55 anos,Feminino,São Paulo,Sim,Sim,Ocasionalmente,['Culturais'],Não,Ocasionalmente,Frequentemente,Sim,5,"Sim, mas não é essencial","Sim, se o preço for acessível","['Agências de Viagem e Operadoras de Turismo',...",Obter Informações Detalhadas,4


Separando dados em características e alvo

In [45]:
X = df.drop('probabilidade_recomendar', axis=1)
y = df['probabilidade_recomendar']

Substituindo os valores 1 e 2 por 0, e 3 e 4 por 1

In [46]:
y = y.replace({1: 0, 2: 0, 3: 1, 4: 1})

Identificar colunas categóricas

In [47]:
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

Preprocessamento

In [48]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    force_int_remainder_cols=False,
    remainder='passthrough'
)

Definição e treinamento do modelo

In [49]:
model = KNeighborsClassifier(n_neighbors=1, weights='uniform')

Criando o pipeline

In [50]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

Divisão dos dados em treino e teste

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Treinando o pipeline

In [52]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['idade', 'genero',
                                                   'municipio_residencia',
                                                   'visita_pontos_turisticos',
                                                   'participa_eventos',
                                                   'frequencia_eventos',
                                                   'tipos_eventos_preferidos',
                                                   'participa_excursões',
                                                   'frequencia_tours_virtuais',
                                                   'frequencia_apps_turismo',
                                                   'usou_apps_turismo',
                                                   'interesse_acessibilidade',
                                                   'pagaria_por_tour_virtual',
                                                   'descoberta_eventos_atracoes',
                                                   'motivacao_uso_app_turismo'])])),
                ('classifier', KNeighborsClassifier(n_neighbors=1))])

Salvando o modelo

In [53]:
joblib.dump(model_pipeline, 'modelo_com_preprocessador.joblib')

['modelo_com_preprocessador.joblib']